In [2]:
import numpy as np
import tensorflow as tf
import math
import random
from matplotlib import pyplot as plt

def sigmoid(x):
    return 1/(1+math.exp(-x))

def sigmoidderivative(x):
    return sigmoid(x) * (1-sigmoid(x))

vec_sigmoid = np.vectorize(sigmoid)
vec_sigmoidderivative = np.vectorize(sigmoidderivative)

In [3]:
class NeuralLayer:

    def __init__(self, numinputs:int, numoutputs:int, activation=None):

        self.numinputs = numinputs
        self.numoutputs = numoutputs
        self.activation = activation
        self.weights = np.random.randn(self.numoutputs, self.numinputs + 1)

    
    def Evaluate(self, inputs):

        inputs = np.append(inputs, np.array([1]))
    
        outputs = self.weights @ inputs # this is \vec{h}

        match self.activation:
            case "Sigmoid":
                outputs = vec_sigmoid(outputs)

            case "Softmax":
                denom = 0
                for i in range(len(outputs)):
                    denom += math.exp[outputs[i]]
                    outputs[i] = math.exp(outputs[i])
                outputs = outputs/denom

            case "ReLU":
                # Put code here, delete the pass
                pass #Alejandro shall not pass
            
            case "Tanh":
                # Put code here, delete the pass
                pass



        return outputs

    
        
    def ComputeLocalGradient(self, inputs):
        # z is output after activation
        # h is output after linear layer
        # w are weights
        # Need to compute three things:
        # dz/dh
        # dh/dw
        # dh/dx

        inputs = np.append(inputs, np.array([1]))
        outputs = self.weights @ inputs


        # This part computes dzdh, and has cases for various activation functions
        match self.activation:
            case "Sigmoid":
                dzdh = np.diag(vec_sigmoidderivative(outputs))
            case "Softmax":
                n = len(outputs)
                dzdh = np.zeros((n, n))
                denom = 0
                for i in range(n):
                    denom += math.exp(outputs[i])
                
                for i in range(n):
                    for j in range(n):
                        if i == j:
                            dzdh[i][j] = (denom * math.exp(outputs[i]) - (math.exp(outputs[i])**2))/(denom**2)
                        else:
                            dzdh[i][j] = -(math.exp(outputs[j]))*(math.exp(outputs[j]))/(denom**2)

            case "ReLU":
                # Put code here and remove pass
                pass
            case "Tanh":
                # Put code here and remove pass
                pass



            

        
        
        
        
        # This part computes dhdw        
        dhdw = np.zeros((self.numoutputs, self.numoutputs, self.numinputs+1)) #because of bias
        for i in range(self.numoutputs):
            for j in range(self.numinputs):
                dhdw[i,i,j] = inputs[i]
            dhdw[i,i,self.numinputs] = 1
            
        # This part computes dhdx
        dhdx = self.weights[:, :-1]


        return (dzdh, dhdw, dhdx)

    
    

        

In [4]:
Layer1 = NeuralLayer(5,3,"Sigmoid")
#Layer1.weights = np.array([[1, 2, -1], [3, -2, 1]])
test1 = np.array([1, 2,3,4,5])
(dzdh, dhdw, dhdx) = Layer1.ComputeLocalGradient(test1)
print(dzdh.shape)
print(dhdw.shape)
print(dhdx.shape)

(3, 3)
(3, 3, 6)
(3, 5)


In [5]:
class NeuralNetwork:

    def __init__(self, errorfunc=None):
        
        self.errorfunc = errorfunc
        self.layers = []
        self.numlayers = 0

    def AppendLayer(self, layer: NeuralLayer):
        # need to check that the new layer to be appended has same 
        # number of inputs as the last layer already in the network
        if len(self.layers) > 0:
            if layer.numinputs == self.layers[-1].numoutputs:
                self.layers.append(layer)
                self.numlayers += 1
            else:
                print("Error: number of inputs does not match previous layer")
        else:
            self.layers.append(layer)
            self.numlayers += 1

        
    def Evaluate(self, inputs):

        outputs = []
        outputs.append(self.layers[0].Evaluate(inputs))

        for i in range(1,self.numlayers):
            outputs.append(self.layers[i].Evaluate(outputs[i-1]))
        
        return outputs

    def ComputeError(self, inputs, trueoutputs):

        outputs = self.Evaluate(inputs)
        
        if self.errorfunc == "MSE":
            n = len(outputs[-1])
            diffs = outputs[-1] - trueoutputs
            err = np.dot(diffs, diffs)
            err = err/(2*n)
            return err

    def BackPropagate(self, inputs, trueoutputs, learningrate):

        outputs = self.Evaluate(inputs)
        gradients = []

        # Compute all the necessary gradients
        for i in range(self.numlayers):
            if i == 0:
                tempinput = inputs
            else:
                tempinput = outputs[i-1]
            
            gradients.append(self.layers[i].ComputeLocalGradient(tempinput))

        match self.errorfunc:
            case "MSE":
                dldz = (0.5) * (outputs[-1] - trueoutputs)

            case "CrossEntropy":
                dldz = np.zeros(len(trueoutputs))
                spot = np.where(1 == trueoutputs)
                dldz[spot] = 1/outputs[-1][spot]
                
            

        # Update weights, working backwards

        currgrad = dldz @ gradients[-1][0]
    
        for i in range(self.numlayers-1, -1, -1):
            self.layers[i].weights -= learningrate * (currgrad @ gradients[i][1])
            currgrad = currgrad @ gradients[i][0] @ gradients[i][2]

In [6]:
MyNN = NeuralNetwork(errorfunc="MSE")
MyLayer1 = NeuralLayer(5, 3, "Sigmoid")
MyLayer2 = NeuralLayer(3, 2, "Sigmoid")


MyNN.AppendLayer(MyLayer1)
MyNN.AppendLayer(MyLayer2)

myinput = np.array([1,2,3,4,5])
mytrue = np.array([1,0])

print(MyNN.ComputeError(myinput, mytrue))

for i in range(10):
    MyNN.BackPropagate(myinput, mytrue, 1)
    print(MyNN.ComputeError(myinput, mytrue))




0.05475984808485329
0.044425528421241436
0.036856884468483966
0.031213273133336498
0.026909518385108064
0.023552582652207502
0.020878883960718863
0.018709246454045758
0.016919473893688846
0.015421641025180333
0.014152193458360059


In [7]:
# Load data from MNIST database
(x_train0, y_train0), (x_test0, y_test0) = tf.keras.datasets.mnist.load_data()
assert x_train0.shape == (60000, 28, 28)
assert x_test0.shape == (10000, 28, 28)
assert y_train0.shape == (60000,)
assert y_test0.shape == (10000,)

# Prepare data for processing
# x_train and x_test need to be reshaped and converted to np.float64
# y_train and y_test need to be one-hot encoded





In [8]:
x_train0[0].reshape(28*28)


array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

In [13]:
MyMNISTNetwork = NeuralNetwork("MSE")
MyMNISTNetwork.AppendLayer(NeuralLayer(28*28,10,"Sigmoid"))


y_train0[0]

#testinput = np.astype(x_train0[0].reshape(28*28), np.float64)
testinput = x_train0[0].reshape(28*28)
testinput = testinput.astype(np.float64)

testinput /= 255.0
#print(testinput.sum())
#print(MyMNISTNetwork.Evaluate(testinput))
#print(MyMNISTNetwork.layers[-1].weights.dtype)

onehot = np.array([0,0,0,0,0,1,0,0,0,0])

print(MyMNISTNetwork.ComputeError(testinput, onehot))
for i in range(100):
    MyMNISTNetwork.BackPropagate(testinput, onehot, 10)
    print(MyMNISTNetwork.ComputeError(testinput, onehot))
    
print(MyMNISTNetwork.ComputeError(testinput, onehot))

print("Final check evaluation: " + str(MyMNISTNetwork.Evaluate(testinput)))

0.29992111279756706
0.29872659842536653
0.29789246407648473
0.29718726984122323
0.29650110057785967
0.2957601628771834
0.29489302319434085
0.29380725948182473
0.29235948562576364
0.2903015382042875
0.2871753356703641
0.2821393154787718
0.27399181634739167
0.26298898185150366
0.25399179804439903
0.2494410281597695
0.2466717321601936
0.24403649060472113
0.24062631752107572
0.23541662013471681
0.22716710446611366
0.21645659371669881
0.20842926097092027
0.20496594901250315
0.20347895827192763
0.2026964137358366
0.20221721730805112
0.2018928998909902
0.20165785564034447
0.20147895600379867
0.20133772831427588
0.20122306410623567
0.2011278769358237
0.20104742725530586
0.20097842038555166
0.2009184914266954
0.2008658964021824
0.20081931926796268
0.20077774699451786
0.2007403862102244
0.20070660608083984
0.20067589824224252
0.20064784810750366
0.2006221139366558
0.2005984113142171
0.20057650146418685
0.20055618233407393
0.2005372817075472
0.20051965182399584
0.20050316513173488
0.2004877109040